In [1]:
import torch 
import torch.nn as nn 
import numpy as np
from collections import OrderedDict

from utils.losses import protop_loss
from utils.sample_parameters import SingleGenerator, HiddenAndKernelGenerator, MonotonicGenerator
from utils.sample_parameters import ParamGenerators
from models.ProtICU import ProtICU
from utils.train_n_test import TrainTest

In [ ]:
x = 

In [2]:
train, val, test = torch.load('data/in-hospital-mortality/tensors.pkl')

In [3]:
# Calculating class weights
perc_mort = np.concatenate((train[1], val[1], test[1])).mean()
class_weights = torch.Tensor([perc_mort, 1-perc_mort])

In [4]:
generators_dict = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': SingleGenerator([128, 256, 512]),
    'EPOCHS': SingleGenerator([20, 40, 50]),
    'OPTIMIZER': SingleGenerator([torch.optim.Adam]),
    'LEARNING_RATE': SingleGenerator([1e-5, 5e-5, 1e-4, 5e-4]),
    'LOSS': SingleGenerator([protop_loss(class_weights, .5 , .5), protop_loss(class_weights, 1, 1),
                             protop_loss(class_weights, .1 , .1), protop_loss(class_weights, 1 , .5),
                             protop_loss(class_weights, .5 , .1), protop_loss(class_weights, .3 , .1)]),
    'EARLY_STOPPING': SingleGenerator([True]),
    'PATIENCE': SingleGenerator(list(range(2,4))),
    'MIN_DELTA': SingleGenerator([5e-5, 1e-4, 5e-4, 1e-3, 5e-3]), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_AND_KERNEL_SIZES': HiddenAndKernelGenerator(range(2,4), [128, 256, 512], [3, 5, 7, 9], 
                               ascending=(True, False)),
    'MAXPOOL': SingleGenerator([2]),
    'OBO_SIZES': MonotonicGenerator(range(1,3), [256, 512], ascending=False),
    'PROTOTYPE_NUM': SingleGenerator([10,20]),
    'DROPOUT': SingleGenerator(np.arange(8)/10)
} # each of these params are sampled INDEPENDENTLY of one another

In [5]:
N = 1
gen = ParamGenerators(generators_dict)
param_samples = gen.sample(N)

In [8]:
stats = []
for i in np.arange(N):
    print(param_samples[i])
    run = TrainTest(ProtICU, (train, val, test), param_samples[i])
    run.train()
    stats.append(run.test())

  0%|          | 0/57 [00:00<?, ?it/s]

{'BATCH_SIZE': 256, 'EPOCHS': 40, 'OPTIMIZER': <class 'torch.optim.adam.Adam'>, 'LEARNING_RATE': 1e-05, 'LOSS': protop_loss_1_1, 'EARLY_STOPPING': True, 'PATIENCE': 3, 'MIN_DELTA': 0.005, 'MAXPOOL': 2, 'OBO_SIZES': array([64]), 'PROTOTYPE_NUM': 10, 'DROPOUT': 0.2, 'HIDDEN_SIZES': array([256, 512]), 'KERNEL_SIZES': array([5, 3])}


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch: 0, train_loss: -0.036058209836483, valid_loss: -0.3903042674064636


  0%|          | 0/57 [00:00<?, ?it/s]

...

Epoch: 17, train_loss: 0.9364207983016968, valid_loss: 0.953764021396637


  0%|          | 0/28 [00:00<?, ?it/s]

val loss increased, patience count:  1
Epoch: 18, train_loss: 0.930306077003479, valid_loss: 0.9540197849273682


100%|██████████| 4/4 [00:00<00:00,  6.78it/s]


Epoch: 19, train_loss: 0.9218802452087402, valid_loss: 0.9494039416313171


In [13]:
import pandas as pd 
df = pd.DataFrame(param_samples)

In [16]:
df.sort_values('auroc', ascending=False) 

,BATCH_SIZE,EPOCHS,OPTIMIZER,LEARNING_RATE,LOSS,EARLY_STOPPING,PATIENCE,MIN_DELTA,MAXPOOL,OBO_SIZES,PROTOTYPE_NUM,DROPOUT,HIDDEN_SIZES,KERNEL_SIZES,epoch_stopped,auroc,auprc,acc,f1
17,128,40,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.5_0.1,True,2,0.00010,2,"[256, 128]",20,0.1,"[128, 256, 512]","[9, 7, 7]",13.0,0.823037,0.471111,0.812441,0.495868
25,128,20,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.3_0.1,True,3,0.00100,2,"[256, 128]",20,0.4,[512],[7],NaN,0.818798,0.455438,0.828761,0.482117
18,256,50,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.5_0.1,True,2,0.00010,2,"[512, 512]",20,0.4,"[64, 128, 256]","[9, 7, 7]",26.0,0.816870,0.477334,0.841296,0.495868
19,128,40,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.1_0.1,True,2,0.00100,2,"[128, 64]",20,0.5,"[128, 256, 512]","[9, 9, 7]",19.0,0.815677,0.459889,0.853832,0.000000
16,128,50,<class 'torch.optim.adam.Adam'>,0.00005,protop_loss_0.3_0.1,True,2,0.00100,2,"[512, 256]",20,0.1,"[64, 128, 256]","[9, 7, 5]",27.0,0.806797,0.448938,0.828997,0.489767
7,512,40,<class 'torch.optim.adam.Adam'>,0.00050,protop_loss_0.3_0.1,True,2,0.00100,2,[64],20,0.3,"[128, 256]","[7, 5]",23.0,0.806796,0.450588,0.847919,0.489277
10,128,20,<class 'torch.optim.adam.Adam'>,0.00010,protop_loss_0.3_0.1,True,3,0.00010,2,[256],20,0.6,"[256, 512]","[3, 3]",NaN,0.800144,0.423244,0.815043,0.461433
3,512,20,<class 'torch.optim.adam.Adam'>,0.00010,protop_loss_0.1_0.1,True,2,0.00500,2,[64],20,0.5,"[128, 256]","[7, 3]",NaN,0.788944,0.411182,0.855014,0.016051
22,512,20,<class 'torch.optim.adam.Adam'>,0.00010,protop_loss_0.5_0.1,True,3,0.00500,2,"[256, 64]",10,0.5,"[64, 128, 256]","[9, 9, 3]",NaN,0.785786,0.404155,0.852176,0.404194
24,128,40,<class 'torch.optim.adam.Adam'>,0.00010,protop_loss_0.3_0.1,True,2,0.00100,2,"[64, 64]",20,0.5,[256],[3],23.0,0.781272,0.402170,0.782167,0.446847


In [9]:
for i in np.arange(N):
    for key,val in stats[i].items():
        param_samples[i][key] = val

In [10]:
torch.save(param_samples, 'results/Proto_experiment_nopush_N'+str(N)+'.pkl')

In [20]:
model = ProtICU(test[0].shape, 2, [64, 128], [5,3], 2, [256, 128], [0,0,0,0,0,1,1,1,1,1], .2)
out, min_dis = model(test[0][:14])

'ProtICU'

In [18]:
check = torch.load('results/Proto_experiment_nopush_N30.pkl')